In [2]:
import pandas as pd

In [3]:
#Read in csv's.
#Need to replace team_id's because this database includes 33 teams (Phoenix and Atlanta had name changes to Arizona and Winnipeg)
#which the database defaulted to calling team_id 52,53,54. I replace them to have team_id's go from 1-31
iter_gp_csv = pd.read_csv(r'C:\Users\SimoS\Documents\NHL\game_plays.csv', iterator = True, chunksize = 1000)
game_plays_df = pd.concat([chunk[(chunk.event == 'Goal')][['game_id','team_id_for', 'team_id_against', 'event']] for chunk in iter_gp_csv])
game_plays_cleaned_df = game_plays_df.replace([52,53,54], [11,27,31])

#Only read in Regular Season games. Again, replace necessary team id's
iter_game_csv = pd.read_csv(r'C:\Users\SimoS\Documents\NHL\game.csv', iterator = True, chunksize = 1000)
game_df = pd.concat([chunk[(chunk.type == 'R')] for chunk in iter_game_csv])
game_cleaned_df = game_df.replace([52,53,54], [11,27,31])

iter_gts_csv = pd.read_csv(r'C:\Users\SimoS\Documents\NHL\game_teams_stats.csv', iterator = True, chunksize = 1000)
game_teams_stats_df = pd.concat([chunk for chunk in iter_gts_csv])
game_teams_stats_cleaned_df = game_teams_stats_df.replace([52,53,54], [11,27,31])

iter_team_csv = pd.read_csv(r'C:\Users\SimoS\Documents\NHL\team_info.csv', iterator = True, chunksize = 1000)
team_df = pd.concat([chunk for chunk in iter_team_csv])
team_cleaned_df = team_df.replace([52,53,54], [11,27,31])

In [9]:
team_cleaned_df.head()

,team_id,franchiseId,shortName,teamName,abbreviation,link
0,1,23,New Jersey,Devils,NJD,/api/v1/teams/1
1,4,16,Philadelphia,Flyers,PHI,/api/v1/teams/4
2,26,14,Los Angeles,Kings,LAK,/api/v1/teams/26
3,14,31,Tampa Bay,Lightning,TBL,/api/v1/teams/14
4,6,6,Boston,Bruins,BOS,/api/v1/teams/6


In [10]:
game_teams_stats_cleaned_df.head()

,game_id,team_id,HoA,won,settled_in,head_coach,goals,shots,hits,pim,powerPlayOpportunities,powerPlayGoals,faceOffWinPercentage,giveaways,takeaways
0,2011030221,1,away,False,OT,Peter DeBoer,3,26,31,12,3,1,44.9,6,7
1,2011030221,4,home,True,OT,Peter Laviolette,4,36,27,6,6,1,55.1,13,4
2,2011030222,1,away,True,REG,Peter DeBoer,4,35,32,12,4,0,50.9,8,7
3,2011030222,4,home,False,REG,Peter Laviolette,1,20,24,32,5,0,49.1,9,6
4,2011030223,4,away,False,OT,Peter Laviolette,3,28,28,4,5,1,50.8,2,1


In [11]:
#These are some simple functions that will be used later in the file.

"""
Return a complete, cleaned teamList
"""
def teamList():
    teamList = team_cleaned_df[['team_id','shortName','teamName']].sort_values("team_id")
    teamList = teamList[(teamList.shortName != 'Phoenix') & (teamList.shortName != 'Atlanta')]
    return teamList
"""
Get the team name given teamID
"""
def getTeamName(userTeamID):
    x = teamList()
    teamName = x[(x.team_id == userTeamID)]
    teamName = teamName[['teamName']]
    return teamName.iloc[0][0]



In [12]:
teamList()

,team_id,shortName,teamName
0,1,New Jersey,Devils
19,2,NY Islanders,Islanders
5,3,NY Rangers,Rangers
1,4,Philadelphia,Flyers
6,5,Pittsburgh,Penguins
4,6,Boston,Bruins
22,7,Buffalo,Sabres
13,8,Montreal,Canadiens
12,9,Ottawa,Senators
20,10,Toronto,Maple Leafs


In [13]:
getTeamName(27)

'Coyotes'

In [14]:
#Get the first goal of each game
goals_df = game_plays_cleaned_df
goals_df = goals_df.drop_duplicates("game_id")
score_first_df = goals_df.merge(game_teams_stats_cleaned_df, on='game_id')
score_first_df = score_first_df[score_first_df.team_id_for == score_first_df.team_id]
score_first_df

,game_id,team_id_for,team_id_against,event,team_id,HoA,won,settled_in,head_coach,goals,shots,hits,pim,powerPlayOpportunities,powerPlayGoals,faceOffWinPercentage,giveaways,takeaways
0,2011030221,1.0,4.0,Goal,1,away,False,OT,Peter DeBoer,3,26,31,12,3,1,44.9,6,7
3,2011030222,4.0,1.0,Goal,4,home,False,REG,Peter Laviolette,1,20,24,32,5,0,49.1,9,6
4,2011030223,4.0,1.0,Goal,4,away,False,OT,Peter Laviolette,3,28,28,4,5,1,50.8,2,1
6,2011030224,4.0,1.0,Goal,4,away,False,REG,Peter Laviolette,2,22,23,10,2,1,62.5,20,5
9,2011030225,4.0,1.0,Goal,4,home,False,REG,Peter Laviolette,1,28,38,8,1,0,45.0,14,7
10,2011030411,26.0,1.0,Goal,26,away,True,OT,Darryl Sutter,2,25,35,4,1,0,55.4,6,4
12,2011030412,26.0,1.0,Goal,26,away,True,OT,Darryl Sutter,2,32,32,8,2,0,53.5,1,3
15,2011030413,26.0,1.0,Goal,26,home,True,REG,Darryl Sutter,4,21,55,12,2,2,56.6,17,5
16,2011030414,1.0,26.0,Goal,1,away,True,REG,Peter DeBoer,3,24,43,8,3,0,31.0,13,2
19,2011030415,1.0,26.0,Goal,1,home,True,REG,Peter DeBoer,2,19,34,6,2,1,52.8,4,3


In [15]:
#See how many times each team scored first in the last 3 years (323 games). Merging with game_df eliminates playoff games.
score_first = pd.merge(score_first_df,game_df, on='game_id')[['game_id','team_id', 'team_id_against','HoA', 'won']]
score_first = score_first[((score_first.game_id.astype(str).str.startswith('2015'))
                            | (score_first.game_id.astype(str).str.startswith('2016')) 
                            | (score_first.game_id.astype(str).str.startswith('2017'))
                            | (score_first.game_id.astype(str).str.startswith('2018')))]
score_first.head(15)

,game_id,team_id,team_id_against,HoA,won
3125,2016020045,16,4.0,home,True
3126,2017020812,7,24.0,home,False
3127,2015020314,11,21.0,home,False
3128,2015020849,12,11.0,home,True
3129,2017020586,24,20.0,home,True
3130,2016020610,15,8.0,away,True
3131,2015020606,26,10.0,home,True
3132,2017020240,23,24.0,away,False
3133,2017020624,21,29.0,home,True
3134,2015020346,22,5.0,away,True


In [16]:
score_first['team_id'].value_counts()

16    189
19    181
5     177
24    174
13    173
6     172
15    172
28    171
29    170
18    169
27    169
2     164
12    164
25    163
10    163
14    163
11    162
30    160
1     160
8     160
3     159
17    157
20    155
21    153
22    152
7     151
9     140
4     137
26    135
23    126
31     83
Name: team_id, dtype: int64

In [17]:
team_df_cleaned = teamList()

In [18]:
score_first_count = score_first['team_id'].value_counts().reset_index()
score_first_count = score_first_count.rename(columns={'team_id':'score_first','index':'team_id'})
score_first_count = score_first_count.merge(team_df_cleaned, on='team_id')
score_first_count 

,team_id,score_first,shortName,teamName
0,16,189,Chicago,Blackhawks
1,19,181,St Louis,Blues
2,5,177,Pittsburgh,Penguins
3,24,174,Anaheim,Ducks
4,13,173,Florida,Panthers
5,6,172,Boston,Bruins
6,15,172,Washington,Capitals
7,28,171,San Jose,Sharks
8,29,170,Columbus,Blue Jackets
9,18,169,Nashville,Predators


In [19]:
home_wins_score_first = score_first[score_first.HoA == 'home']['won'].value_counts()
home_wins_score_first

True     1840
False     731
Name: won, dtype: int64

In [20]:
1840/(1840+731)

0.7156748346946713

In [21]:
away_wins_score_first = score_first[score_first.HoA == 'away']['won'].value_counts()
away_wins_score_first


True     1494
False     859
Name: won, dtype: int64

In [22]:
1494/(1494+859)

0.6349341266468338

### Scoring first at home yielded a 71.57% win rate, while scoring first away yielded a 63.49% win rate.

Let's format a training dataframe which will include:
- game id 
- scorefirst %
- head-to-head
- avg shots per team
- and the target outcome

In [32]:
#Create train set (2015, 2016-2017, 2017-2018, 2018-2019 seasons)
data_1 = game_cleaned_df[['game_id', 'home_team_id', 'away_team_id', 'outcome']]
data_1 = data_1[((data_1.game_id.astype(str).str.startswith('2015')) 
                 | (data_1.game_id.astype(str).str.startswith('2016')) 
                    | (data_1.game_id.astype(str).str.startswith('2017')) 
                    | (data_1.game_id.astype(str).str.startswith('2018')))]

In [34]:
data_1.loc[data_1.outcome.str.contains('home'), 'outcome'] = 1
data_1.loc[data_1.outcome.str.contains('away',na=False), 'outcome'] = 0
data_1.head()

,game_id,home_team_id,away_team_id,outcome
3885,2016020045,16,4,1
3886,2017020812,7,24,0
3887,2015020314,11,21,0
3888,2015020849,12,11,1
3889,2017020586,24,20,1


### 1 implies the home team won, 0 implies home team lost.

I will now create a function to calculate head_to_head stats of each team for each game played since 2015

In [37]:
#Get head to head stats of the last 3 seasons
g = game_cleaned_df[((game_df.type == "R") & ((game_df.game_id.astype(str).str.startswith('2015')) 
                                      | (game_df.game_id.astype(str).str.startswith('2016')) 
                                      | (game_df.game_id.astype(str).str.startswith('2017')) 
                                        | (game_df.game_id.astype(str).str.startswith('2018'))))][['home_team_id', 'away_team_id', 'outcome']]

"""
This function is used to get the head_to_head stats of two teams.
"""
def train_head_to_head_aux(userTeam1, userTeam2):
    g_2 = g[(((g.home_team_id == userTeam2) &  (g.outcome.str.contains('home',na=False)))
             | ((g.away_team_id == userTeam2) & (g.outcome.str.contains('away',na=False))))]
    count = g_2['away_team_id'].value_counts()
    count2 = g_2['home_team_id'].value_counts()
    y = pd.concat([count, count2], axis = 1).reset_index()
    #Fix the NaN's
    y = y.fillna(0)
    userTeam2_name = getTeamName(userTeam2) + '_wins'
    y[userTeam2_name] = y['away_team_id'] + y['home_team_id']
    y[userTeam2_name] = pd.to_numeric(y[userTeam2_name], downcast='integer')
    y['away_team_id'] = pd.to_numeric(y['away_team_id'], downcast='integer')
    y = y.rename(columns={"away_team_id":"away_wins", "home_team_id":"home_wins","index":"team_id"})
    y = y[y.team_id == userTeam1]
    if y.empty == True:
        return 0
    return y.iloc[0][3]

The above head_to_head function returns the head to head value of each team played. See an example below:

In [38]:
train_head_to_head_aux(1,2)

11

In [39]:
train_head_to_head_aux(2,1)

5.0

Above shows that Team 1 head to head with Team 2 is an 11-5 record. Meaning Team 1 has beat Team 2 6 more times than Team 2 beating Team 1. Using our head_to_head function above, we can calculate a quotient in wins between each team. If Vegas has won 4 games against Chicago, and Chicago has won 2 against Vegas, it will give Vegas a 0.67 (4/6 = 2/3). The next operation will take some time, as it calculates the head to head of each game since 2015

In [41]:
home_team_list = data_1.home_team_id.to_list()
away_team_list = data_1.away_team_id.to_list()
count = 0
hth_list = []
for x in home_team_list:
    y = away_team_list[count]
    away = train_head_to_head_aux(y,x)
    home = train_head_to_head_aux(x, y)
    if away == 0:
        hth_list.append(1)
    else:
        hth_list.append(home / (home+away))
    count+=1


In [43]:
data_1['head_to_head'] = hth_list
data_1.head()

,game_id,home_team_id,away_team_id,outcome,head_to_head
3885,2016020045,16,4,1,0.750000
3886,2017020812,7,24,0,0.375000
3887,2015020314,11,21,0,0.526316
3888,2015020849,12,11,1,0.625000
3889,2017020586,24,20,1,0.277778


In [45]:
data_2 = data_1.merge(score_first_count, left_on='home_team_id', right_on='team_id')
data_2['home_score_first'] = data_2['score_first'] / 323 #divide by 323 because there are 323 games played since 2015
data_2 = data_2.drop(columns=['team_id','score_first','shortName','teamName'])


,game_id,home_team_id,away_team_id,outcome,head_to_head,teamName,home_score_first
0,2016020045,16,4,1,0.750000,Blackhawks,0.585139
1,2017021255,16,19,0,0.473684,Blackhawks,0.585139
2,2015020170,16,26,1,0.750000,Blackhawks,0.585139
3,2015020002,16,3,0,0.375000,Blackhawks,0.585139
4,2015020832,16,24,0,0.500000,Blackhawks,0.585139


In [48]:
data_2.head()

,game_id,home_team_id,away_team_id,outcome,head_to_head,home_score_first
0,2016020045,16,4,1,0.750000,0.585139
1,2017021255,16,19,0,0.473684,0.585139
2,2015020170,16,26,1,0.750000,0.585139
3,2015020002,16,3,0,0.375000,0.585139
4,2015020832,16,24,0,0.500000,0.585139


In [49]:
data_3 = data_2.merge(score_first_count, left_on='away_team_id', right_on='team_id')
data_3['away_score_first'] = data_3['score_first'] / 323
data_3.head()

,game_id,home_team_id,away_team_id,outcome,head_to_head,home_score_first,team_id,score_first,shortName,teamName,away_score_first
0,2016020045,16,4,1,0.750000,0.585139,4,137,Philadelphia,Flyers,0.424149
1,2017020181,16,4,1,0.750000,0.585139,4,137,Philadelphia,Flyers,0.424149
2,2015021048,16,4,0,0.750000,0.585139,4,137,Philadelphia,Flyers,0.424149
3,2018021142,16,4,0,0.750000,0.585139,4,137,Philadelphia,Flyers,0.424149
4,2017020539,7,4,1,0.583333,0.467492,4,137,Philadelphia,Flyers,0.424149


In [51]:
data_3 = data_3.drop(columns=['team_id','score_first','shortName','teamName'],axis=1)

In [52]:
data_3['%_score_first'] = (data_3['home_score_first'] / (data_3['away_score_first']+ data_3['home_score_first']))
data_3.head()

,game_id,home_team_id,away_team_id,outcome,head_to_head,home_score_first,away_score_first,%_score_first
0,2016020045,16,4,1,0.750000,0.585139,0.424149,0.579755
1,2017020181,16,4,1,0.750000,0.585139,0.424149,0.579755
2,2015021048,16,4,0,0.750000,0.585139,0.424149,0.579755
3,2018021142,16,4,0,0.750000,0.585139,0.424149,0.579755
4,2017020539,7,4,1,0.583333,0.467492,0.424149,0.524306


In [55]:
data_4 = data_3.drop(columns = ['game_id','home_team_id','away_team_id','home_score_first','away_score_first'])

In [56]:
data_4.head()

,outcome,head_to_head,%_score_first
0,1,0.750000,0.579755
1,1,0.750000,0.579755
2,0,0.750000,0.579755
3,0,0.750000,0.579755
4,1,0.583333,0.524306


In [74]:
from sklearn.linear_model import LogisticRegression  # for Logistic Regression algorithm
from sklearn.model_selection import train_test_split #to split the dataset for training and testing
from sklearn.neighbors import KNeighborsClassifier  # for K nearest neighbours
from sklearn import svm  #for Support Vector Machine (SVM) Algorithm
from sklearn import metrics #for checking the model accuracy
from sklearn.tree import DecisionTreeClassifier #for using Decision Tree Algoithm
from sklearn.preprocessing import StandardScaler

In [75]:
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import numpy as np
sns.set()

### Scale inputs

In [76]:
y = data_4['outcome']
x1 = data_4[['head_to_head', '%_score_first']]

In [77]:
scaler = StandardScaler()
scaler.fit(x1)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [78]:
x_scaled = scaler.transform(x1)

In [79]:
x_scaled

array([[ 1.48472856,  1.88754513],
       [ 1.48472856,  1.88754513],
       [ 1.48472856,  1.88754513],
       ...,
       [ 1.48472856, -1.76247153],
       [ 1.48472856, -1.76247153],
       [ 1.48472856, -1.76247153]])

In [80]:
train, test = train_test_split(data_4, test_size = 0.3)# in this our main data is split into train

In [81]:
train_X = train[['head_to_head','%_score_first']]
train_y = train.outcome

In [82]:
test_X=test[['head_to_head','%_score_first']]
test_y = test.outcome

In [83]:
model = svm.SVC() #select the algorithm
model.fit(train_X,train_y) # we train the algorithm with the training data and the training output
prediction=model.predict(test_X) #now we pass the testing data to the trained algorithm
print('The accuracy of the SVM is:',metrics.accuracy_score(prediction,test_y))#now we check the accuracy of the algorithm. 
#we pass the predicted output by the model and the actual output

c:\users\simos\appdata\local\programs\python\python36-32\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


The accuracy of the SVM is: 0.6342438374417055


In [84]:
#Logistic regression
model = LogisticRegression()
model.fit(train_X,train_y)
prediction=model.predict(test_X)
print('The accuracy of the Logistic Regression is',metrics.accuracy_score(prediction,test_y))

The accuracy of the Logistic Regression is 0.6502331778814124


c:\users\simos\appdata\local\programs\python\python36-32\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


# The Logistic Regression has 65.02% accuracy

In [85]:
def confusion_matrix(data,actual_values,model):
        
        # Confusion matrix 
        
        # Parameters
        # ----------
        # data: data frame or array
            # data is a data frame formatted in the same way as your input data (without the actual values)
            # e.g. const, var1, var2, etc. Order is very important!
        # actual_values: data frame or array
            # These are the actual values from the test_data
            # In the case of a logistic regression, it should be a single column with 0s and 1s
            
        # model: a LogitResults object
            # this is the variable where you have the fitted model 
            # e.g. results_log in this course
        # ----------
        
        #Predict the values using the Logit model
        pred_values = model.predict(data)
        # Specify the bins 
        bins=np.array([0,0.5,1])
        # Create a histogram, where if values are between 0 and 0.5 tell will be considered 0
        # if they are between 0.5 and 1, they will be considered 1
        cm = np.histogram2d(actual_values, pred_values, bins=bins)[0]
        # Calculate the accuracy
        accuracy = (cm[0,0]+cm[1,1])/cm.sum()
        # Return the confusion matrix and 
        return cm, accuracy

In [86]:
confusion_matrix(test_X,test_y,model)

(array([[369., 294.],
        [231., 607.]]), 0.6502331778814124)